In [2]:
import pandas as pd
import requests
import random
import time
from pytz import timezone
import tqdm


import os

In [ ]:

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import re

In [3]:
nikkei255 = pd.read_csv('./nikkei255.csv', index_col=0)

In [4]:
nikkei255.head()

,name,sector
code,,
2002,日清製粉グループ本社,食品
2269,明治ホールディングス,食品
2282,日本ハム,食品
2501,サッポロホールディングス,食品
2502,アサヒグループホールディングス,食品


## Set up Selenium

In [5]:
os.environ['CSVDIR'] + '/daily'

'C:\\Users\\y509541\\git\\AITradingND\\Japan Stock\\data/daily'

In [5]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import re

options = Options()
options.binary_location = '/usr/bin/firefox'
#options.add_argument('-headless')
profile = webdriver.FirefoxProfile()
profile.set_preference('browser.download.folderList', 2) # custom location
profile.set_preference('browser.download.manager.showWhenStarting', False)
profile.set_preference('browser.download.dir', os.environ['CSVDIR'] + '/daily')
profile.set_preference("browser.download.manager.alertOnEXEOpen", False)
profile.set_preference("browser.download.manager.closeWhenDone", True)
profile.set_preference("browser.download.manager.focusWhenStarting", False)
#application/octet-stream,application/vnd.ms-excel 
profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/csv')
driver = webdriver.Firefox(firefox_profile=profile, options=options)

## Access login page

In [6]:
driver.get('https://www.yahoo.co.jp/')
source = driver.page_source
soup = BeautifulSoup(source,'html.parser')
element = driver.find_element_by_link_text("ログイン")
element.click()
time.sleep(2)

### Load user credentials

In [7]:
credentials = {}
with open("credential.txt") as myfile:
    for line in myfile:
        name, var = line.partition("=")[::2]
        credentials[name.strip()] = var.rstrip('\n')

### type user name and move forward

In [8]:
element = driver.find_element_by_id("username")
element.send_keys(credentials['username'])
driver.find_element_by_id("btnNext").click()
time.sleep(2)

### type password ang move forward

In [9]:
element = driver.find_element_by_id("passwd")
element.send_keys(credentials["passwd"])
driver.find_element_by_id("btnSubmit").click()
time.sleep(2)

## Open finance page

In [10]:
driver.get('http://finance.yahoo.co.jp')

In [11]:
driver.set_page_load_timeout(10)

In [12]:
#GET /quote/2002.T/history/download HTTP/2
Host = 'https://finance.yahoo.co.jp'


In [15]:
# Yahoo Finance
# 日付	始値	高値	安値	終値	出来高	調整後終値*
# Zipzline Ingest CSV
# date,open,high,low,close,volume,dividend,split
def formatDownloadedCSV(csvFile):
    rows = []
    firstLine = True
    with open(filename, encoding="shift-jis") as f:
        lines = f.readlines()
        for line in lines:
            if firstLine:
                firstLine = False
            else:

                # you may also want to remove whitespace characters like `\n` at the end of each line
                items = line.rstrip().split(',')
                try:
                    rows.append({'date': f'{items[0]}',
                                'open': f'{items[1]}',
                                'high': f'{items[2]}',
                                'low': f'{items[3]}',
                                'close': f'{items[6]}',
                                'volume': f'{items[5]}',
                                'dividend': 0.0,
                                'split': 1.0
                                })
                except:
                    pass

    df = pd.DataFrame(rows)
    dayColumn = pd.to_datetime(df['date']).dt.strftime('%Y/%m/%d')
    df = df.set_index(pd.DatetimeIndex(dayColumn)).drop('date', axis = 1)

    return df

In [18]:
%%time
for code, company in nikkei255.iterrows():

    name = company['name']
    print(f'{code}: {name}')
    try:
        driver.get(Host + f'/quote/{code}.T/history/download')
    except:
        pass

    filename = os.environ['CSVDIR'] + '/daily/' + f'{code}.T.csv'
    df = formatDownloadedCSV(filename)
    df.to_csv(filename)
    
    time.sleep(random.randint(5, 15))


4568: 第一三共
4578: 大塚ホールディングス
5019: 出光興産
5020: ENEOSホールディングス
5101: 横浜ゴム
5108: ブリヂストン
5201: AGC
5202: 日本板硝子
5214: 日本電気硝子
5232: 住友大阪セメント
5233: 太平洋セメント
5301: 東海カーボン
5332: TOTO
5333: 日本碍子
5401: 日本製鉄
5406: 神戸製鋼所
5411: JFEホールディングス
5541: 大平洋金属
3436: SUMCO
5703: 日本軽金属ホールディングス
5706: 三井金属鉱業
5707: 東邦亜鉛
5711: 三菱マテリアル
5713: 住友金属鉱山
5714: DOWAホールディングス
5801: 古河電気工業
5802: 住友電気工業
5803: フジクラ
5901: 東洋製罐グループホールディングス
5631: 日本製鋼所
6103: オークマ
6113: アマダ
6301: 小松製作所
6302: 住友重機械工業
6305: 日立建機
6326: クボタ
6361: 荏原製作所
6367: ダイキン工業
6471: 日本精工
6472: NTN
6473: ジェイテクト
7004: 日立造船
7011: 三菱重工業
7013: IHI
3105: 日清紡ホールディングス
6479: ミネベアミツミ
6501: 日立製作所
6503: 三菱電機
6504: 富士電機
6506: 安川電機
6645: オムロン
6674: ジーエス・ユアサ コーポレーション
6701: 日本電気
6702: 富士通
6703: 沖電気工業
6724: セイコーエプソン
6752: パナソニック
6753: シャープ
6758: ソニー
6762: TDK
6770: アルプスアルパイン
6841: 横河電機
6857: アドバンテスト
6902: デンソー
6952: カシオ計算機
6954: ファナック
6971: 京セラ
6976: 太陽誘電
7735: SCREENホールディングス
7751: キヤノン
7752: リコー
8035: 東京エレクトロン
7003: 三井E&Sホールディングス
7012: 川崎重工業
7201: 日産自動車
7202: いすゞ自動車
7203: トヨタ自動車
72

In [19]:
driver.get("https://login.yahoo.co.jp/config/login?logout=1")
driver.quit()

In [11]:
import glob
path = os.path.join(os.environ['CSVDIR'] ,'daily')

for code, company in nikkei255.iterrows():
    os.rename(os.path.join(path, str(code) + '.T.csv'), os.path.join(path, str(code) + '.csv'))

In [29]:
arr =[]
for code, company in nikkei255.iterrows():
    df = pd.read_csv(f'./data/daily/{code}.csv')
    date = df.tail()['date'].values[0]
    arr.append({'code': code, 'company': company, 'date': date})
    print(f'{code}: {company} begin date :{date}')
    #df.to_csv(f'./data/{code}.csv')
df = pd.DataFrame(arr)

502, dtype: object begin date :2019-01-10
2503: name      キリンホールディングス
sector             食品
Name: 2503, dtype: object begin date :2019-01-10
2531: name      宝ホールディングス
sector           食品
Name: 2531, dtype: object begin date :2019-01-10
2801: name      キッコーマン
sector        食品
Name: 2801, dtype: object begin date :2019-01-10
2802: name      味の素
sector     食品
Name: 2802, dtype: object begin date :2019-01-10
2871: name      ニチレイ
sector      食品
Name: 2871, dtype: object begin date :2019-01-10
2914: name      日本たばこ産業
sector         食品
Name: 2914, dtype: object begin date :2019-01-10
3101: name      東洋紡
sector     繊維
Name: 3101, dtype: object begin date :2019-01-10
3103: name      ユニチカ
sector      繊維
Name: 3103, dtype: object begin date :2019-01-10
3401: name      帝人
sector    繊維
Name: 3401, dtype: object begin date :2019-01-10
3402: name      東レ
sector    繊維
Name: 3402, dtype: object begin date :2019-01-10
3861: name      王子ホールディングス
sector         パルプ・紙
Name: 3861, dtype: object begin date :

In [26]:
for code, company in nikkei255.iterrows():
    df = pd.read_csv(os.path.join(path, str(code) + '.csv'))
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['date'] > dt.datetime(2019,1,1)]
    df = df.set_index('date')
    df.to_csv(os.path.join(path, str(code) + '.csv'))